In [1]:
import numpy as np
import pandas as pd

In [2]:
def read_file(filename):
    with open(filename, 'r') as f:
        content = f.read()
        
    return content

In [3]:
def load_data():
    data = {}
    for index in range(3):
        X_train = pd.read_csv('Xtr%s_mat50.csv' % index, delimiter=' ', names=range(0,50)).values
        seq_train = pd.read_csv('Xtr%s.csv' % index, names='1').values
        X_test = pd.read_csv('Xte%s_mat50.csv' % index, delimiter=' ', names=range(0,50)).values
        seq_test = pd.read_csv('Xte%s.csv' % index, names='1').values
        labels = pd.read_csv('Ytr%s.csv' % index, names=('Id', 'Bound'), skiprows=[0], delimiter=',')

        data[index] = {
            'seq_train': seq_train,
            'X_train': X_train,
            'y_train': labels['Bound'].values,
            'seq_test': seq_test,
            'X_test': X_test,
            'ids': labels['Id'].values,
        }
        
    return data

DATA = load_data()

In [4]:
from data_manipulation import split_train_test_valid, get_precision

## Kernel ridge regression 

In [6]:
train_X, train_y, test_X, test_y, valid_X, valid_y = split_train_test_valid(DATA[0]['X_train'], DATA[0]['y_train'])
train_X.shape

(1200, 50)

In [7]:
from ridge_regression import get_ridge_prediction, kernel_ridge_regression

In [8]:
def linear_K(x1, x2):
    return np.matmul(x1.T, x2)

In [11]:
REG_PARAMS_SPAN = [10**i for i in range(-10, 10)]

def get_best_reg_param():
    test_precisions = []
    for reg in REG_PARAMS_SPAN:
        alpha = kernel_ridge_regression(linear_K, train_X, train_y, reg)
        pred = get_ridge_prediction(linear_K, train_X, test_X, alpha)
        test_precisions.append(get_precision(pred, test_y))
        
    best_reg_index = max(range(len(REG_PARAMS_SPAN)), key=lambda x: test_precisions[x])
    return REG_PARAMS_SPAN[best_reg_index]

In [12]:
reg_param = get_best_reg_param()
alpha = kernel_ridge_regression(linear_K, train_X, train_y, reg_param)
pred = get_ridge_prediction(linear_K, train_X, valid_X, alpha)
get_precision(pred, valid_y)

0.59250000000000003

## Substring Kernel 

In [19]:
train_X, train_y, test_X, test_y, valid_X, valid_y = split_train_test_valid(DATA[0]['seq_train'], DATA[0]['y_train'])
train_X.shape

(1200, 1)

In [20]:
from substring_kernel import K_k, substring_kernel

In [21]:
LAMBDA = 0.1
K_k(LAMBDA, 2, 'car', 'car') - 2*LAMBDA**4+LAMBDA**6 < 10**-5

True

In [22]:
K_k(0.1, 5, 
   'CAGCTTTTATCACCTTTGAGGGAAAGTCATATTAATTTAATACTGCACACACTTGTACAACAGATCTTCTTTACTATTAAAACTCAGTTTATCAAATCACA',
   'AATAACATACCCCACTCTTTCATCTCAATCAAAAATTGAAAAAGTCAAAGAATCCTGCTTTTTTGTTTTTCTCCAAGCCATTACCCCCTCTTGATCATTGC'
   )

3.1540274232800625e-09

In [23]:
# This is too slow by several orders of magnitude...
for j in range(train_X.shape[0]):
    print(j)
    K_k(0.1, 2, train_X[1,0], train_X[j,0])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34


KeyboardInterrupt: 

## Final results

In [10]:
def produce_results(preds, ids):
    data = [
        '%s,%s' % (id, pred) for pred, id in zip(preds, ids)
    ]
    
    data.insert(0, 'Ids,Bound')
    with open('submission.csv', 'w') as f:
        f.write('\n'.join(data))

In [14]:
def train_tune_and_pred_on_test():
    preds = []
    ids = []
    for k in range(3):
        train_X, train_y, test_X, test_y, valid_X, valid_y = split_train_test_valid(DATA[k]['X_train'], DATA[k]['y_train'])

        reg_param = get_best_reg_param()
        
        # Validation precision
        reg_param = get_best_reg_param()
        alpha = kernel_ridge_regression(linear_K, train_X, train_y, reg_param)
        pred = get_ridge_prediction(linear_K, train_X, valid_X, alpha)
        print("Dataset %s has found a parameter with validation precision %.3f" % (k, get_precision(pred, valid_y)))
        
        alpha = kernel_ridge_regression(linear_K, DATA[k]['X_train'], DATA[k]['y_train'], reg_param)
        pred = get_ridge_prediction(linear_K, DATA[k]['X_train'], DATA[k]['X_test'], alpha)
        
        preds += list(pred)

    produce_results(preds, range(3000))

In [15]:
train_tune_and_pred_on_test()

Dataset 0 has found a parameter with validation precision 0.613
Dataset 1 has found a parameter with validation precision 0.775
Dataset 2 has found a parameter with validation precision 0.570
